In [ ]:
from ceas.utils import cleanup_applicants,transform_list_to_dummies
import ceas.config as cfg
import jinja2
import pandas as pd
import datetime
import holidays

In [3]:
dfs = pd.read_pickle(cfg.INTERIM_DATA_DIR/"dfs.pkl")

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
dfs['applicants']['subjects'].value_counts()

subjects
Historia, geografía y Ciencias Sociales                                                  38
Inglés                                                                                   23
Educación Física y Salud                                                                 20
Lenguaje y Comunicación                                                                  19
Artes Musicales                                                                          11
Biología, Ciencias Naturales                                                              8
Educadora de Párvulos                                                                     6
Matemática                                                                                4
Biología, Química                                                                         3
Artes Visuales                                                                            3
Física (CN), Matemática                                                

In [7]:
cleaned_applicants = cleanup_applicants(dfs['applicants'])

df = cleaned_applicants.copy()
df = transform_list_to_dummies(df, 'subjects', prefix='s')
df


,applicant_id,timestamp,email,first_name,last_name,second_last_name,rut,phone,comuna_residencia,genero,ed_licence_level,subjects,university,undergrad_year,available_days,max_hours_per_week,complementary_information,cv_link,qualified_to_work_at_schools_cert,phone_raw,anios_egreso,subjects_raw,ed_licence_level_raw,s_Biología,s_Ciencias Naturales,s_Educación Física y Salud,"s_Historia, geografía y Ciencias Sociales",s_Religión,s_Tecnología,s_TP - Mecánica Automotriz,s_Artes Musicales,s_Orientación,s_Lenguaje y Comunicación,s_Artes Visuales,s_Inglés,s_Matemática,s_Física (CN),s_Filosofía,s_Química,s_TP - Contabilidad
0,1.0,3/2/2025 11:35:42,veronica.salas2017@umce.cl,Verónica Alessandra,Salas,Rosales,NaN,+56978475925,Peñalolén,Femenino,[Media],"[Biología, Ciencias Naturales]",U. Metropolitana De Ciencias De La Educación,0023-03-28,"[Monday, Tuesday, Wednesday, Thursday, Friday]",40.0,NaN,https://drive.google.com/open?id=1mve8InHCX0TU...,https://drive.google.com/open?id=19TmZjd65jXIS...,+56978475925,NaN,"Biología, Ciencias Naturales",Educación Media [7° a IV medio],1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.0,3/2/2025 12:09:41,alemoyahernandez@gmail.com,Alejandra Beatriz,Moya,Hernandez,NaN,+56979991492,Peñalolén,Femenino,[Inicial (PreKinder/Kinder)],[],U. Andrés Bello,2020-12-22,"[Monday, Tuesday, Wednesday, Thursday, Friday]",40.0,Disponibilidad inmediata y al 100% jornada lab...,https://drive.google.com/open?id=1jTe31idCdjCu...,https://drive.google.com/open?id=1gyHIamTQppOT...,+56979991492,5.0,Educadora de Párvulos,Educación Parvularia [Kínder y Pre kínder],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.0,3/2/2025 12:22:39,janoqueo@gmail.com,Alejandro Vicente,Caqueo,Romero,NaN,NaN,Peñalolén,Masculino,[Media],[Educación Física y Salud],U. Metropolitana De Ciencias De La Educación,1997-12-31,"[Monday, Tuesday, Wednesday, Thursday, Friday]",25.0,"Mi horario libre es solo en la mañana, hasta l...",https://drive.google.com/open?id=1uUdJudm_45Wm...,https://drive.google.com/open?id=1nEcjddWledl3...,56985022787,28.0,Educación Física y Salud,Educación Media [7° a IV medio],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4.0,3/2/2025 12:48:46,yeraycamposhistoria@gmail.com,Yeray Campos,Campos,Gracia,NaN,+56954148120,Vitacura,Masculino,[Media],"[Historia, geografía y Ciencias Sociales]",Otra,2021-09-01,"[Monday, Tuesday, Wednesday, Thursday, Friday]",44.0,NaN,https://drive.google.com/open?id=1bnOTcIoHugZi...,https://drive.google.com/open?id=1tTOb6iry5AB0...,+56954148120,4.0,"Historia, geografía y Ciencias Sociales",Educación Media [7° a IV medio],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5.0,3/2/2025 13:19:38,nmoralespizarro@gmail.com,Nicolas Morales,Morales,Pizarro,NaN,NaN,San Bernardo,Masculino,[Básica],[],U. Academia De Humanismo Cristiano,2016-02-03,"[Monday, Tuesday, Wednesday, Thursday, Friday]",40.0,NaN,https://drive.google.com/open?id=1Efxy9e-t9f1a...,https://drive.google.com/open?id=1SWGJ8sriXbxW...,56950844043,9.0,Convivencia escolar,Educación Básica con mención [1° a 6° Básico],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,205.0,20/3/2025 17:53:35,seeba.guuti10@gmail.com,Sebastian,Gutierrez,Alarcon,20357775-3,NaN,Recoleta,Masculino,"[Básica, Inicial (PreKinder/Kinder), Media]",[Educación Física y Salud],U. San Sebastián,2024-03-20,"[Monday, Tuesday, Wednesday, Thursday, Friday]",44.0,Tengo disponibilidad inmediata quiero comenzar...,https://drive.google.com/open?id=1l7QxiQgHTKbm...,https://drive.google.com/open?id=1F0CDcMPlRfBa...,56967145027,1.0,Educación Física y Salud,"Educación Parvularia [Kínder y Pre kínder], Ed...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
205,206.0,20/3/2025 19:41:21,pipeklauser@gmail.com,Luis Felipe,Silva,Klauser,20749451-8,+56978071257,Estación Central,Masculino,[Media],"[Historia, geografía y Ciencias Sociales]",U. Academia De Humanismo Cristiano,1994-06-24,"[Monday, Tuesday, Wednesday, Th

In [ ]:
cleaned_applicants[['ed_licence_level_raw']].value_counts()


In [ ]:
cleaned_applicants['ed_licence_level'].value_counts()

In [ ]:
cleaned_applicants[cleaned_applicants['subjects'].apply(lambda x: len(x))==0][['ed_licence_level','subjects','subjects_raw','complementary_information']]

In [ ]:
dfs['applicants'][['phone','comuna_residencia','ed_licence_level','subjects','undergrad_year','available_days']].to_csv(cfg.INTERIM_DATA_DIR/"applicants_select_cols.csv", index=False)

In [ ]:
pd.read_pickle(cfg.PROJ_ROOT/"request.pickle")

In [ ]:
session_state = {}

In [ ]:
# Guarda el nombre de la app en session_state para que los managers lean la hoja correcta 
session_state["app_name"] = "appReemplazos"
session_state['sheet_names'] = ["Users", "Schools", "Applicants", "Candidates", "Requests", 
                                   "RequestCandidates","Receptions", "UserValidations"]#
# sheet_names es una concatenación de app_name + sheet_name
session_state['sheet_names'] = [session_state["app_name"] + sheet_name for sheet_name in session_state['sheet_names']]

# definimos los roles que pueden tener los usuarios: ["owner", "admin", "oficina_central", "admin_colegio", "user_colegio"] y luego
# pasamos a una serie de pandas que tenga un orden categórico para poder comparar los roles

session_state['roles'] = ["owner", "admin", "oficina_central", "admin_colegio", "user_colegio"]
# ahora lo transformamos a una categoría ordenada
# session_state['roles_rank'] = {role: i for i, role in enumerate(session_state['roles'])}

session_state['roles_rank'] = {
    "owner": 0,
    "admin": 1,
    "oficina_central": 2,
    "admin_colegio": 3,
    "user_colegio": 4
}

session_state['niveles_educativos'] = ["Inicial (PreKinder/Kinder)", "Básica", "Media", "Técnico Profesional", "Educación Diferencial"]

session_state['cursos_por_nivel_educativo'] = {
    "Inicial (PreKinder/Kinder)": ["PreKinder", "Kinder"],
    "Básica": [f"{i}° Básico" for i in range(1, 9)],
    "Media": [f"{i}° Medio" for i in range(1, 5)],
    "Técnico Profesional": [f"{i}° Medio" for i in range(3, 5)],
    "Educación Diferencial": ["PreKinder", "Kinder"] + [f"{i}° Básico" for i in range(1, 9)] + [f"{i}° Medio" for i in range(1, 5)]
}

session_state['asignaturas_por_nivel_educativo'] = {
    "Inicial (PreKinder/Kinder)": [""],# sin asignaturas
    "Básica": ["Generalista", "Lenguaje y Comunicación", "Matemáticas", "Ciencias Naturales", "Historia, Geografía y Ciencias Sociales", "Educación Física y Salud", "Artes Visuales", "Artes Musicales", "Tecnología","Orientación", "Religión", "Inglés"],
    "Media": ["Lenguaje y Comunicación", "Matemáticas", "Ciencias Naturales", "Historia, Geografía y Ciencias Sociales", "Educación Física y Salud", "Artes Visuales", "Artes Musicales", "Tecnología", "Orientación", "Religión", "Filosofía", "Inglés", "Física", "Química", "Biología"],
    "Técnico Profesional": ["Electricidad","Mecánica Automotriz","Construcción","Hotelería","Gastronomía","Atención de Párvulos","Administración","Telecomunicaciones","Enfermería","Laboratorio Clínico","Contabilidad"],
    "Educación Diferencial": [""]

}

In [ ]:
df_applicants = dfs['applicants'].copy()
df_requests = dfs['requests'].copy()

In [ ]:
str_dia = df_requests['dias_seleccionados'].iloc[0]

In [ ]:
# split the string into a list
str_dia_m = str_dia.replace("[","").replace("]", "").split("), ")
str_dia_m

In [ ]:
request_dict = pd.read_pickle(cfg.INTERIM_DATA_DIR/"request_dict.pkl")

In [ ]:
type(request_dict['fecha_inicio'])

In [ ]:
type(request_dict['dias_seleccionados'][0])

In [ ]:

import datetime
import ast
def validate_request(new_request):
    """
    Validate a new replacement request record according to general, obvious criteria.
    Returns (is_valid: bool, error_messages: list) so the caller can handle them.

    new_request is expected to be a dict or Series-like object with fields:
      - nivel_educativo (list or None)
      - asignatura (dict or list, or None)
      - curso (dict or list, or None)
      - fecha_inicio (datetime.date or None)
      - fecha_fin (datetime.date or None)
      - dias_seleccionados (list of weekdays or None)
      - horarios_seleccionados (dict or None)
      - genero (str)
      - anios_egreso (int)
      - disponibilidad (str)
      - candidato_preferido (str)
      - otras_preferencias (str)
      - comentarios (str)
      - status (str)
      - created_at (datetime)

    Example checks:
      1. fecha_inicio <= fecha_fin
      2. nivel_educativo / asignatura / curso must not be empty
      3. dias_seleccionados / horarios no empty if there's a range of dates
      4. status in an allowed set (e.g. ["creada","aprobada","rechazada"]?)

    We'll be as general as possible. Adjust or expand as needed.
    """
    # execute adjusted_types to convert data that was read as a string


    errors = []
    # 1) Check date range
    fi = new_request.get("fecha_inicio")
    ff = new_request.get("fecha_fin")
    if fi is None or ff is None:
        errors.append("Fechas de inicio/fin no definidas.")
    else:
        if fi > ff:
            errors.append("La fecha de inicio no puede ser posterior a la fecha de fin.")

    # 2) Check that there's at least one 'nivel_educativo'
    nivel_educ = new_request.get("nivel_educativo", [])
    if not nivel_educ or len(nivel_educ) == 0:
        errors.append("Debe seleccionar al menos un nivel educativo.")

    # 3) Check that asignatura is not empty
    #   new_request["asignatura"] = {nivel -> list} or list
    asig = new_request.get("asignatura", {})
    # assume it might be a dict or a list
    total_asig = 0
    if isinstance(asig, dict):
        for niv, arr in asig.items():
            total_asig += len(arr)
    elif isinstance(asig, list):
        total_asig += len(asig)
    if total_asig == 0:
        errors.append("Debe especificar al menos una asignatura para el reemplazo.")

    # 4) Check that 'curso' is not empty similarly
    curso_val = new_request.get("curso", {})
    total_cursos = 0
    if isinstance(curso_val, dict):
        for niv, arr in curso_val.items():
            total_cursos += len(arr)
    elif isinstance(curso_val, list):
        total_cursos += len(curso_val)
    if total_cursos == 0:
        errors.append("Debe seleccionar al menos uno o varios cursos/niveles.")

    # 5) If we have a date range, presumably we have dias_seleccionados
    dias = new_request.get("dias_seleccionados", [])
    if (fi is not None and ff is not None) and (fi <= ff):
        if not dias:
            errors.append("No se detectaron días entre las fechas seleccionadas, verifique la selección.")

    # 6) Check for 'status'
    valid_statuses = ["creada","aprobada","rechazada","finalizada"]
    status_val = new_request.get("status", "")
    if status_val not in valid_statuses:
        errors.append(f"Estado (status) inválido: {status_val} (debe estar en {valid_statuses}).")

    # 7) We might also check generically if "horarios_seleccionados" is consistent with the days
    horarios = new_request.get("horarios_seleccionados", {})
    if dias and not horarios:
        errors.append("Existen días seleccionados, pero no se definieron horarios.")
    # optional advanced checks: each 'dia' in dias is in horarios, etc.

    # 8) check basic field presence
    if not new_request.get("school_name"):
        errors.append("Falta school_name en la solicitud.")
    if not new_request.get("created_by"):
        errors.append("Falta created_by en la solicitud.")

    is_valid = (len(errors) == 0)
    return (is_valid, errors)


In [ ]:
isValid,errors =  validate_request(request_dict)
print("isValid:", isValid)
print("errors:", errors)

In [ ]:
from ceas.serialize_data import serialize_request_for_sheets, deserialize_request_from_sheets

# testeamos serialización primero

serialized_request = serialize_request_for_sheets(request_dict)
print("Serialized request:", serialized_request)

In [ ]:
# veamos los tipos de cada campo
print("Serialized request types:")
for key, value in serialized_request.items():
    print("\n")
    print("--------------------------------")
    print(f"{key}: {value}")
    print(f"{key}: {type(value)}")
    print("--------------------------------")
    

In [ ]:

def filter_applicants_by_request(new_request, df_applicants):
    """
    Given a new request dictionary (as validated above) and a DataFrame 'df_applicants'
    that contains the 'applicants' table or an equivalent derived table with available candidates,
    apply the relevant filters to return a subset that matches the request's criteria.

    Typical filters:
      - Genero if != "Indiferente"
      - For each 'nivel_educativo', we might check the applicant's ed_licence_level
      - For each 'asignatura' needed, we might check applicant's 'subjects'
      - If date range implies certain days (and request demands "Completa" availability),
        we only want applicants whose available_days covers all those days

    Adjust or expand the logic as needed. This is a general skeleton.
    """
    subset = df_applicants.copy()

    # 1. Genero filter (if request['genero'] != "Indiferente")
    genero_req = new_request.get("genero", "Indiferente")
    if genero_req != "Indiferente":
        subset = subset[ subset["genero"] == genero_req ]

    # 2. ed_licence_level filter
    # request has e.g. new_request["nivel_educativo"] = ["Media","Básica",...]
    nivel_req = new_request.get("nivel_educativo", [])
    if nivel_req:
        # check if applicant's ed_licence_level has at least one of the requested
        subset = subset[
            subset["ed_licence_level"].apply(
                lambda app_levels: any(niv in app_levels for niv in nivel_req)
            )
        ]

    # 3. asignaturas
    # new_request["asignatura"] = {nivel -> list}, or list
    # We'll do a simplified approach: flatten all assigned subjects
    requested_subjs = []
    asig_val = new_request.get("asignatura", {})
    if isinstance(asig_val, dict):
        for k,arr in asig_val.items():
            requested_subjs.extend(arr)
    elif isinstance(asig_val, list):
        requested_subjs = asig_val
    requested_subjs = list(set(requested_subjs))  # unique

    if requested_subjs:
        # filter where applicant's "subjects" intersects with requested_subjs
        subset = subset[
            subset["subjects"].apply(
                lambda s: any(subj in s for subj in requested_subjs)
            )
        ]

    # 4. Disponibilidad
    # If "Completa" => applicant must have all days in new_request["dias_seleccionados"].
    # If "Parcial" => at least one day in common.
    disp_type = new_request.get("disponibilidad", "Parcial")  # default "Parcial"?
    req_days = new_request.get("dias_seleccionados", [])
    if req_days:
        if disp_type == "Completa":
            # only those applicants whose available_days contains all req_days
            subset = subset[
                subset["available_days"].apply(
                    lambda app_days: all(day in app_days for day in req_days)
                )
            ]
        else:
            # "Parcial" => intersection not empty
            subset = subset[
                subset["available_days"].apply(
                    lambda app_days: any(day in app_days for day in req_days)
                )
            ]

    # 5. anios_egreso => e.g. 'undergrad_year' is a DATETIME => check difference from current date
    # if we assume "anios_egreso" means minimum years since they graduated
    anios_min = new_request.get("anios_egreso", 0)
    if anios_min > 0:
        import datetime
        now = datetime.datetime.now().date()
        def years_since(grad_date):
            # grad_date is e.g. an approx date in applicant's 'undergrad_year'
            if grad_date is None:
                return 0
            return now.year - grad_date.year
        subset = subset[
            subset["undergrad_year"].apply(lambda d: years_since(d) >= anios_min)
        ]

    # More filters can be appended as needed

    return subset

In [ ]:
request = pd.read_pickle(cfg.PROJ_ROOT/"request.pickle")


In [ ]:
filter_applicants_by_request(request, df_applicants)

In [ ]:
validate_request(request_dict)

In [ ]:
# vamos a extraer un registro de la tabla df_requests como un diccionario, con cada key siendo el nombre de la columna
# y el valor siendo el valor de la columna para ese registro
request = df_requests.to_dict(orient="records")[0]

In [ ]:
# chequeamos el tipo de cada registro:
for key in request.keys():
    print(f"{key}: {type(request[key])}")

In [ ]:
new_request = request.copy()
new_request

In [ ]:
# chequeamos el tipo de cada registro de df_applicants
for i in df_applicants.columns:
    print(f"{i}: {df_applicants[i].apply(type).value_counts().reset_index().values}")

In [ ]:
pd.set_option('display.max_rows', None)
df_applicants.head()

In [ ]:
asig = new_request.get("asignatura", {})
total_asig = 0
if isinstance(asig, dict):
    for niv, arr in asig.items():
        total_asig += len(arr)
elif isinstance(asig, list):
    total_asig += len(asig)
if total_asig == 0:
    print("Debe especificar al menos una asignatura para el reemplazo.")

In [ ]:
# convertir el string a un dict
import ast
ast.literal_eval(new_request["asignatura"])

In [ ]:
validate_request(request)